In [78]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split

from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #choose whether to use gpu or cpu

In [80]:
# data_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder'  #path to the folder containing the images
# data_dir = r'C:\Users\Public\PartIIB project 2023_2024\PastData\helical_size'
first_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\first_half\first_quarter'
second_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\first_half\second_quarter'
third_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\second_half\third_quarter'
fourth_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\second_half\fourth_quarter'


In [81]:
# multi-input dataset
class CustomImageDataset(Dataset):
    def __init__(self, first_dir, second_dir, third_dir, fourth_dir, transform=None):
        self.first_dir = first_dir
        self.second_dir = second_dir
        self.third_dir = third_dir
        self.fourth_dir = fourth_dir
        self.transform = transform
        # self.images1 = sorted(os.listdir(first_dir))
        self.images1 = sorted(os.listdir(first_dir), key=self.custom_sort_key)
        self.images2 = sorted(os.listdir(second_dir), key=self.custom_sort_key)
        self.images3 = sorted(os.listdir(third_dir), key=self.custom_sort_key)
        self.images4 = sorted(os.listdir(fourth_dir), key=self.custom_sort_key)
        self.labels1 = [self.extract_label(img) for img in self.images1]
        self.labels2 = [self.extract_label(img) for img in self.images2]
        self.labels3 = [self.extract_label(img) for img in self.images3]
        self.labels4 = [self.extract_label(img) for img in self.images4]
        
        #input verification
        assert len(self.images1) == len(self.images2) == len(self.images3) == len(self.images4)

        self.length = len(self.images1)

    def __len__(self):
        return self.length
        # return len(self.images1)

    def __getitem__(self, idx):
        img_name1 = os.path.join(self.first_dir, self.images1[idx])
        image1 = Image.open(img_name1)
        img_name2 = os.path.join(self.second_dir, self.images2[idx])
        image2 = Image.open(img_name2)
        img_name3 = os.path.join(self.third_dir, self.images3[idx])
        image3 = Image.open(img_name3)
        img_name4 = os.path.join(self.fourth_dir, self.images4[idx])
        image4 = Image.open(img_name4)
    
        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)
            image3 = self.transform(image3)
            image4 = self.transform(image4)
        
        # print(self.labels1)
        # print(self.labels2)
        # print(self.labels3)
        # print(self.labels4)

        label1 = self.labels1[idx]
        label2 = self.labels2[idx]
        label3 = self.labels3[idx]
        label4 = self.labels4[idx]
        return image1, image2, image3, image4, label1, label2, label3, label4

    def custom_sort_key(self, item):
        # Extract numbers after 'Fig_' and 't-'
        fig_number = float(item.split('Fig_')[1].split('__')[0])
        t_number = float(item.split('t-')[1].split('_')[0])
        
        return fig_number, t_number
    
    def extract_label(self, img_name):
        # Assuming that the label is part of the filename before the first underscore
        label = float(img_name[-17:-5]) #this is the right code
        # label = str(img_name)
        return label

In [82]:
data_transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.45), (0.25))]) 

custom_dataset = CustomImageDataset(first_dir=first_dir, second_dir=second_dir, third_dir=third_dir, fourth_dir=fourth_dir, transform=data_transform)

# # Accessing the data
# for img, label in custom_dataset:
#     print(f"Image shape: {img.shape}, Label: {label}")

print(len(custom_dataset))

# train_set, val_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.15), int(len(custom_dataset)*0.100056)]) #splits data into training, validation and test sets
train_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.25013)])
print(len(train_set))
# print(len(val_set))
print(len(test_set))

3950
2962
988


In [83]:
#hyperparameters
num_epochs = 30
batch_size = 1
learning_rate = 0.0005

# allData = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)
# # print(allData)
# for (images1, images2, images3, images4, labels1, labels2, labels3, labels4) in allData:
#     print(labels1, labels2, labels3, labels4)


train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
for (images1, images2, images3, images4, labels1, labels2, labels3, labels4) in train:
    print(labels1.item(), labels2.item(), labels3.item(), labels4.item())
    break

21.49329 6.343753 10.03911 11.13706


In [117]:
class ConvNet(nn.Module): # note need to find out image size
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,30,20, padding='same') #in_channels, out_channels, kernel_size
        self.normalise1 = nn.BatchNorm2d(30)
        # self.pool = nn.MaxPool2d(5,5) #kernel_size, stride (shift x pixel to the right)
        self.pool1 = nn.AvgPool2d(10, stride=10)
        # self.pool1 = nn.MaxPool2d(10, stride=10)
        self.conv2 = nn.Conv2d(30, 30, 20, padding='same')
        # self.normalise2 = nn.BatchNorm2d(16)
        # self.pool2 = nn.AvgPool2d(2, stride=2)
        # self.pool2 = nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(30, 30, 10, padding='same')
        # self.normalise3 = nn.BatchNorm2d(32) 
        self.conv4 = nn.Conv2d(30, 30, 10, padding='same')
        # self.fc1 = nn.Linear(16*3*3, 120) # 3x3 is the size of the image after 2 conv layers, 16 is the number of channels, 120 is the number of nodes in the hidden layer
        # self.fc2 = nn.Linear(120,84)
        # self.fc3 = nn.Linear(60, 1)
        self.fc0 = nn.Linear(30, 10)
        self.fc = nn.Linear(32*5*5, 1)
        self.fc1 = nn.Linear(32*5*5, 400)
        self.fc2 = nn.Linear(400,200)
        self.fc3 = nn.Linear(200,4)
        self.dropout = nn.Dropout(0.1)


    def forward(self, x, y, z, a):
        print(x.shape)
        x = self.pool1(F.relu(self.normalise1(self.conv1(x)))) 
        x = self.pool1(F.relu(self.normalise1(self.conv2(x)))) 
        x = x.view(-1, 30)  #flatten

        x = self.fc0(x)
        print(x.shape)
        y = self.pool1(F.relu(self.normalise1(self.conv1(y)))) 
        y = self.pool1(F.relu(self.normalise1(self.conv2(y)))) 
        print(y.shape)
        y = y.view(-1, 30)  #flatten
        y = self.fc0(y)
        print(y.shape)
        z = self.pool1(F.relu(self.normalise1(self.conv1(z)))) 
        z = z.view(-1, 30)  #flatten
        z = self.fc0(z)
        a = self.pool1(F.relu(self.normalise1(self.conv1(a)))) 
        a = a.view(-1, 30)  #flatten
        a = self.fc0(a)

        combined = torch.cat((x,y,z,a), dim=-1)
        print(combined.shape)
        combined = self.pool1(F.relu(self.normalise1(self.conv3(combined)))) 
        combined = self.pool1(F.relu(self.normalise1(self.conv4(combined)))) 
        combined = combined.view(-1, 32*5*5)  #flatten
        combined = F.relu(self.fc1(combined))
        combined = F.relu(self.fc2(combined))
        combined = self.fc3(combined)
        return combined

In [118]:
model = ConvNet().to(device)

# loss and optimizer
criterion = nn.MSELoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
#training loop
n_total_steps = len(train)
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images1, images2, images3, images4, labels1, labels2, labels3, labels4) in enumerate(train):
        images1 = images1.to(device)
        images2 = images2.to(device)
        images3 = images3.to(device)    
        images4 = images4.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        labels4 = labels4.to(device)

        #forward
        outputs = model(images1, images2, images3, images4)
        # print(labels)
        labels1 = labels1.float() 
        labels2 = labels2.float()
        labels3 = labels3.float()
        labels4 = labels4.float()
        labels = torch.cat((labels1, labels2, labels3, labels4), dim=1)
        loss = criterion(outputs, labels)

        #backward
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # if (i+1) % 1000 ==0:
        # print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
        total_loss += loss.item()
    print(f'epoch {epoch+1}/{num_epochs}, average loss = {total_loss/len(train):.4f}')
            


torch.Size([1, 1, 100, 100])
torch.Size([1, 10])
torch.Size([1, 30, 1, 1])
torch.Size([1, 10])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 100 for tensor number 2 in the list.